<a href="https://colab.research.google.com/github/RP335/microtonal_experiments/blob/main/microtonal_parrot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generator for 180 midi files for the dataset. different kinds of edo classes and only triads.

In [72]:
!sudo apt-get install python@3.11.7
!python3 --version
!pip install mido
!apt-get install fluidsynth
!pip install midi2audio
!pip install music21
# !pip install xenharmlib


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python@3.11.7
E: Couldn't find any package by glob 'python@3.11.7'
E: Couldn't find any package by regex 'python@3.11.7'
Python 3.10.12
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fluidsynth is already the newest version (2.2.5-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


installs

In [73]:
import mido
import os
from midi2audio import FluidSynth
import librosa
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from music21 import stream, note, midi, pitch
# import xenharmlib as xh
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV




In [93]:
# !rm -rf microtonal_audio_files microtonal_midi_files

In [58]:
# this requires 3.11.7 or greater which doesn't work with colab
# from xenharmlib import EDOTuning, export, UpDownNotation

# edos = [31, 19, 11]
# for edo in edos:
#     edoTuning = EDOTuning(edo)
#     n_edo = UpDownNotation(edoTuning)
#     chord = n_edo.note_scale(
#         [n_edo.note(s, 4) for s in ['C', 'E', 'G', 'Bb']]
#     )
#     filename = f'microtonal_chord_{edo}edo.wav'
#     export.audio.export_wav(filename, chord, duration = 2.0, play_as_chord= True, sample_rate=22500)


imports

In [59]:

# # Paths
# soundfont_path = '/usr/share/sounds/sf2/FluidR3_GM.sf2'
# midi_dir = 'microtonal_midi_files'
# output_dir_audio = 'microtonal_audio_files'

# os.makedirs(output_dir_audio, exist_ok=True)

# # Soundfont initialization
# fs = FluidSynth(soundfont_path)

# # Midi 2 audio
# for edo in edos:
#     for i in range(num_files_per_edo):
#         midi_file = os.path.join(midi_dir, f'{edo}edo_chord_{i+1}.mid')
#         audio_file = os.path.join(output_dir_audio, f'{edo}edo_chord_{i+1}.wav')

#         if os.path.exists(midi_file):
#             fs.midi_to_audio(midi_file, audio_file)
#         else:
#             print(f"no file exists like this: {midi_file}")

# print(f'Converted all MIDI in d {midi_dir} directory.')


In [94]:
audio_dir = 'microtonal_audio_files'

def extract_features(file_name):
    y, sr = librosa.load(file_name, sr=22050)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    return np.mean(mfccs.T, axis=0)

features = []
labels = []

for file_name in os.listdir(audio_dir):
    if file_name.endswith('.wav'):
        file_path = os.path.join(audio_dir, file_name)
        edo_label = file_name.split('_')[2].replace('edo', '')
        feature = extract_features(file_path)
        features.append(feature)
        labels.append(edo_label)

# Convert to DataFrame
df = pd.DataFrame(features)
df['label'] = labels

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(df.drop('label', axis=1))
y = df['label']

In [95]:
# Load dataset
# dataset = pd.read_csv('microtonal_dataset.csv')
# X = dataset.drop(columns=['label']).values
# y = dataset['label'].values

# Encode labels
# le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Reshape X for CNN input (single channel audio)
# X = X.reshape(X.shape[0], X.shape[1], 1)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

model = tf.keras.models.Sequential()

# TDNN layers (1D Convolutional layers)
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))

# LSTM layer
model.add(tf.keras.layers.LSTM(128, return_sequences=True))
model.add(tf.keras.layers.LSTM(64))

# Dense layers
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(np.unique(y_encoded)), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')


Epoch 1/200
6/6 [==============================] - 8s 419ms/step - loss: 1.8076 - accuracy: 0.1302 - val_loss: 1.7966 - val_accuracy: 0.0833
Epoch 2/200
6/6 [==============================] - 0s 68ms/step - loss: 1.7642 - accuracy: 0.2604 - val_loss: 1.7946 - val_accuracy: 0.2083
Epoch 3/200
6/6 [==============================] - 0s 68ms/step - loss: 1.7332 - accuracy: 0.2552 - val_loss: 1.7938 - val_accuracy: 0.1458
Epoch 4/200
6/6 [==============================] - 0s 65ms/step - loss: 1.6960 - accuracy: 0.2865 - val_loss: 1.7952 - val_accuracy: 0.1250
Epoch 5/200
6/6 [==============================] - 0s 56ms/step - loss: 1.6706 - accuracy: 0.3021 - val_loss: 1.7952 - val_accuracy: 0.1250
Epoch 6/200
6/6 [==============================] - 0s 60ms/step - loss: 1.6007 - accuracy: 0.3750 - val_loss: 1.7967 - val_accuracy: 0.1458
Epoch 7/200
6/6 [==============================] - 0s 62ms/step - loss: 1.5529 - accuracy: 0.3646 - val_loss: 1.7972 - val_accuracy: 0.0833
Epoch 8/200
6/6 [==

In [96]:
audio_dir = 'microtonal_audio_files'

def extract_features(file_name):
    y, sr = librosa.load(file_name, sr=22050)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    return np.mean(mfccs.T, axis=0)

features = []
labels = []

for file_name in os.listdir(audio_dir):
    if file_name.endswith('.wav'):
        file_path = os.path.join(audio_dir, file_name)
        edo_label = file_name.split('_')[2].replace('edo', '')
        feature = extract_features(file_path)
        features.append(feature)
        labels.append(edo_label)

# Convert to DataFrame
df = pd.DataFrame(features)
df['label'] = labels

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(df.drop('label', axis=1))
y = df['label']

In [97]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.6875


In [98]:

# Define the directory containing audio files
audio_dir = 'microtonal_audio_files'

# Function to extract MFCC features from an audio file
def extract_features(file_name):
    y, sr = librosa.load(file_name, sr=22050)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    return np.mean(mfccs.T, axis=0)

# Initialize lists to store features and labels
features = []
labels = []

# Loop through the audio files and extract features
for file_name in os.listdir(audio_dir):
    if file_name.endswith('.wav'):
        file_path = os.path.join(audio_dir, file_name)
        edo_label = file_name.split('_')[2].replace('edo', '')
        feature = extract_features(file_path)
        features.append(feature)
        labels.append(edo_label)

# Convert to DataFrame
df = pd.DataFrame(features)
df['label'] = labels
print(df)

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(df.drop('label', axis=1))
y = df['label']



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define different classifiers
classifiers = {
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier(),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier()
}

# Train and evaluate each classifier
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{name} Accuracy: {accuracy}')



# Define parameter grid for RandomForest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize grid search
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit grid search
grid_search.fit(X_train, y_train)

# Best parameters
best_params = grid_search.best_params_
print(f'Best parameters: {best_params}')

# Train model with best parameters
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred_best = best_model.predict(X_test)

# Calculate accuracy
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f'Accuracy with best parameters: {accuracy_best}')


def extract_additional_features(file_name):
    y, sr = librosa.load(file_name, sr=22050)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    return np.hstack([
        np.mean(mfccs.T, axis=0),
        np.mean(chroma.T, axis=0),
        np.mean(spectral_contrast.T, axis=0)
    ])

# Initialize lists to store features and labels
features = []
labels = []

# Loop through the audio files and extract features
for file_name in os.listdir(audio_dir):
    if file_name.endswith('.wav'):
        file_path = os.path.join(audio_dir, file_name)
        edo_label = file_name.split('edo')[1].split('_')[0]
        feature = extract_additional_features(file_path)
        features.append(feature)
        labels.append(edo_label)

# Convert to DataFrame
df = pd.DataFrame(features)
df['label'] = labels

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(df.drop('label', axis=1))
y = df['label']

# Split the data and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy with additional features: {accuracy}')





              0           1          2          3           4          5  \
0   -465.873444   97.099113  34.580509 -36.366459  -85.648315 -96.146225   
1   -474.893860   99.703484  76.178375  42.816307    5.778016 -29.174362   
2   -475.888214   96.545090  66.200989  24.863632  -17.921068 -53.659142   
3   -491.940735   79.698509  68.718697  52.626957   33.460762  12.877384   
4   -484.151276   92.047607  80.361740  63.192101   42.646355  20.474701   
..          ...         ...        ...        ...         ...        ...   
235 -484.151276   92.047607  80.361740  63.192101   42.646355  20.474701   
236 -488.358398   90.804031  75.538185  53.390949   27.562714   0.874036   
237 -460.084503   92.595779  13.230347 -65.658829 -102.074226 -83.015587   
238 -464.872223  103.637939  64.860168  13.680273  -36.025364 -72.539940   
239 -464.117157   88.634331   5.926870 -72.476700 -102.268822 -73.735321   

             6          7          8          9  ...         31         32  \
0   -69.2

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best parameters: {'max_depth': 30, 'min_samples_split': 5, 'n_estimators': 100}
Accuracy with best parameters: 0.6875
Accuracy with additional features: 1.0
